<a href="https://colab.research.google.com/github/ajagota7/Shaping/blob/main/Lifegate_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import numpy as np
import os
from google.colab import drive
import pickle
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import minimize
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import torch

# deadend dependencies

In [3]:
!git clone https://github.com/microsoft/med-deadend.git


Cloning into 'med-deadend'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 130 (delta 52), reused 77 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 395.48 KiB | 10.99 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# shaping dependencies

In [4]:
!git clone https://github.com/ajagota7/Shaping.git

Cloning into 'Shaping'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 81 (delta 36), reused 67 (delta 24), pack-reused 0
Receiving objects: 100% (81/81), 11.54 MiB | 30.06 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [5]:
# %cd /content/Shaping

In [6]:
# !git pull origin main

In [7]:
# cd /content/

In [8]:
# %cd /content/Shaping

import zipfile

with zipfile.ZipFile('/content/Shaping/lifegate_1.zip', 'r') as zip_ref:
    # zip_ref.extractall('/content/med-deadend/lifegate/results/lifegate_1')
    zip_ref.extractall('/content/Shaping/')

In [9]:
%cd /content/med-deadend/lifegate


# results_dir = 'results/lifegate_1/'
results_dir = '/content/Shaping/'
# Load the Q tables from the primary learning agent, Q_D and Q_R value functions
with open(results_dir+'tabular_qnet.pkl', 'rb') as fq:
    ai = pickle.load(fq)

with open(results_dir+'tabular_qd.pkl', 'rb') as fd:
    ai_d = pickle.load(fd)

with open(results_dir+'tabular_qr.pkl', 'rb') as fr:
    ai_r = pickle.load(fr)

/content/med-deadend/lifegate


In [10]:
q_table = np.zeros((10, 10, 5))
q_d = np.zeros_like(q_table)
q_r = np.zeros_like(q_table)


for i in range(10):
    for j in range(10):
        for a in range(5):
            key = tuple([j, i, a])
            try:
                q_table[i,j,a] = ai.q[key]
                q_d[i,j,a] = ai_d.q[key]
                q_r[i,j,a] = ai_r.q[key]
            except:
                pass

In [11]:
import yaml
import random

In [12]:
from lifegate import LifeGate

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
params = yaml.safe_load(open('config.yaml', 'r'))

In [14]:
np.random.seed(seed=params['random_seed'])
random.seed(params['random_seed'])
random_state = np.random.RandomState(params['random_seed'])

In [15]:
env = LifeGate(max_steps=params['episode_max_len'], state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.1)

In [16]:
env_30 = LifeGate(max_steps=30, state_mode='tabular',
                        rendering=True, image_saving=False, render_dir=None, rng=random_state, death_drag = 0.1)

In [17]:
import Shaping

In [18]:
from Shaping import *

In [19]:
%cd /content/Shaping

from choose_actions import action_probs_top_n_epsilon
from shaping_features import *
from gen_policies import *
from IS import *
from subset_policies import *
from v_pi_e import *
from optimization import *
from neural_net import *
from prep_variance import *
from SCOPE_variance import SCOPE_variance

/content/Shaping


# Generating policies

## interm updates

In [ ]:
def experiment_actions(nb_episodes, env, action_probs):
    """
    Run the experiment for a specified number of episodes.

    Parameters:
    - nb_episodes: Number of episodes
    - env: Experiment environment
    - action_probs: Action probabilities

    Returns:
    - policies: List of policies (pi_b or pi_e)
    """
    # Define the dtype for the structured array
    dtype = [
        ('state_last', np.float64, (2,)),
        ('action', np.int64),
        ('reward', np.float64),
        ('state', np.float64, (2,)),
        ('timestep', np.int64),
        ('psi', np.float64)
    ]

    policies = []
    for i in range(nb_episodes):
        trajectory = np.empty(0, dtype=dtype)
        s = env.reset()
        env.render()
        term = False
        timestep = 0
        while not term:
            state_last = s
            print(s)
            action = choose_action(tuple(s), action_probs)
            s, r, term, _ = env.step(action)

            psi = smallest_distance_to_deadend(state_last, env)
            data_point = np.array([(state_last, action, r, s, timestep, psi)], dtype=dtype)
            trajectory = np.append(trajectory, data_point)
            timestep += 1

        policies.append(trajectory)

    with open('policies.pkl', 'wb') as f:
        pickle.dump(policies, f)

    return policies

In [ ]:
P_pi_b_good = np.zeros((10, 10, 5))

P_pi_b_good[2,9,3] = 1
P_pi_b_good[1,9,3] = 1
P_pi_b_good[0,9,1] = 1
P_pi_b_good[0,8,1] = 1
P_pi_b_good[0,7,1] = 1
P_pi_b_good[0,6,1] = 1
P_pi_b_good[0,5,1] = 1
P_pi_b_good[0,4,4] = 1
P_pi_b_good[1,4,4] = 1
P_pi_b_good[2,4,4] = 1
P_pi_b_good[3,4,4] = 1
P_pi_b_good[4,4,4] = 1
P_pi_b_good[5,4,1] = 1
P_pi_b_good[5,5,1] = 1
P_pi_b_good[5,4,1] = 1
P_pi_b_good[5,3,1] = 1
P_pi_b_good[5,2,1] = 1
P_pi_b_good[5,1,1] = 1

In [ ]:
P_pi_b_good_stoch = np.zeros((10, 10, 5))

P_pi_b_good_stoch[2,9,3] = 0.5
P_pi_b_good_stoch[2,9,1] = 0.5

P_pi_b_good_stoch[2,8,3] = 0.5
P_pi_b_good_stoch[2,8,1] = 0.5

P_pi_b_good_stoch[2,7,3] = 0.5
P_pi_b_good_stoch[2,7,1] = 0.5

P_pi_b_good_stoch[1,9,3] = 0.5
P_pi_b_good_stoch[1,9,1] = 0.5

P_pi_b_good_stoch[1,8,3] = 0.5
P_pi_b_good_stoch[1,8,1] = 0.5

P_pi_b_good_stoch[1,7,3] = 0.5
P_pi_b_good_stoch[1,7,1] = 0.5

P_pi_b_good_stoch[2,6,3] = 1
P_pi_b_good_stoch[1,6,3] = 1
P_pi_b_good_stoch[1,5,3] = 1


P_pi_b_good_stoch[0,9,1] = 1
P_pi_b_good_stoch[0,8,1] = 1
P_pi_b_good_stoch[0,7,1] = 1
P_pi_b_good_stoch[0,6,1] = 1
P_pi_b_good_stoch[0,5,1] = 1

P_pi_b_good_stoch[0,4,1] = 0.5
P_pi_b_good_stoch[0,4,4] = 0.5
P_pi_b_good_stoch[0,4,1] = 0.5
P_pi_b_good_stoch[0,4,4] = 0.5
P_pi_b_good_stoch[0,3,1] = 0.5
P_pi_b_good_stoch[0,3,4] = 0.5
P_pi_b_good_stoch[0,2,1] = 0.5
P_pi_b_good_stoch[0,2,4] = 0.5

P_pi_b_good_stoch[1,4,1] = 0.5
P_pi_b_good_stoch[1,4,4] = 0.5
P_pi_b_good_stoch[1,4,1] = 0.5
P_pi_b_good_stoch[1,4,4] = 0.5
P_pi_b_good_stoch[1,3,1] = 0.5
P_pi_b_good_stoch[1,3,4] = 0.5
P_pi_b_good_stoch[1,2,1] = 0.5
P_pi_b_good_stoch[1,2,4] = 0.5

P_pi_b_good_stoch[2,4,1] = 0.5
P_pi_b_good_stoch[2,4,4] = 0.5
P_pi_b_good_stoch[2,4,1] = 0.5
P_pi_b_good_stoch[2,4,4] = 0.5
P_pi_b_good_stoch[2,3,1] = 0.5
P_pi_b_good_stoch[2,3,4] = 0.5
P_pi_b_good_stoch[2,2,1] = 0.5
P_pi_b_good_stoch[2,2,4] = 0.5

P_pi_b_good_stoch[3,4,1] = 0.5
P_pi_b_good_stoch[3,4,4] = 0.5
P_pi_b_good_stoch[3,4,1] = 0.5
P_pi_b_good_stoch[3,4,4] = 0.5
P_pi_b_good_stoch[3,3,1] = 0.5
P_pi_b_good_stoch[3,3,4] = 0.5
P_pi_b_good_stoch[3,2,1] = 0.5
P_pi_b_good_stoch[3,2,4] = 0.5

P_pi_b_good_stoch[4,4,1] = 0.5
P_pi_b_good_stoch[4,4,4] = 0.5
P_pi_b_good_stoch[4,4,1] = 0.5
P_pi_b_good_stoch[4,4,4] = 0.5
P_pi_b_good_stoch[4,3,1] = 0.5
P_pi_b_good_stoch[4,3,4] = 0.5
P_pi_b_good_stoch[4,2,1] = 0.5
P_pi_b_good_stoch[4,2,4] = 0.5

P_pi_b_good_stoch[0,1,4] = 1
P_pi_b_good_stoch[1,1,4] = 1
P_pi_b_good_stoch[2,1,4] = 1
P_pi_b_good_stoch[3,1,4] = 1
P_pi_b_good_stoch[4,1,4] = 1

P_pi_b_good_stoch[5,4,1] = 1
P_pi_b_good_stoch[5,3,1] = 1
P_pi_b_good_stoch[5,2,1] = 1
P_pi_b_good_stoch[5,1,1] = 1



In [ ]:
P_pi_b_bad = np.zeros((10, 10, 5))

P_pi_b_bad[2,9,4] = 1

P_pi_b_bad[3,9,1] = 1
P_pi_b_bad[3,8,1] = 1
P_pi_b_bad[3,7,1] = 1
P_pi_b_bad[3,6,4] = 1

P_pi_b_bad[4,6,2] = 1
P_pi_b_bad[4,7,2] = 1
P_pi_b_bad[4,8,2] = 1
P_pi_b_bad[4,9,4] = 1

P_pi_b_bad[5,9,1] = 1
P_pi_b_bad[5,8,1] = 1
P_pi_b_bad[5,7,1] = 1
P_pi_b_bad[5,6,4] = 1

P_pi_b_bad[6,6,4] = 1
P_pi_b_bad[7,6,4] = 1
P_pi_b_bad[8,6,4] = 1






In [ ]:
env.main_deaths

[[9, 0],
 [9, 1],
 [9, 2],
 [9, 3],
 [9, 4],
 [9, 5],
 [9, 6],
 [9, 7],
 [9, 8],
 [9, 9],
 [8, 0]]

# Gen Policies

In [20]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env, P_pi_e)

In [21]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(1000, env_30, P_pi_b)

P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(1000, env_30, P_pi_e)

# Prep

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [ ]:
test1 = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [ ]:
# IS_tensor, samples_IS, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor = test1.prepare()

# Modified class with bootstrap_all_terms
IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor = test1.prepare()

<ipython-input-20-827856c56c37>:89: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)


In [ ]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = test1.prep_policies()

In [ ]:
states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)


# sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)
samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)


In [ ]:
sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first
IS_SCOPE_1 = IS_tensor*sum_terms

In [ ]:
SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

E_IS_SCOPE_2 = torch.mean(torch.mean(samples_IS_SCOPE, dim =1))
E_IS_E_SCOPE_2 = torch.mean(torch.mean(samples_IS,dim = 1 )) * torch.mean(torch.mean(samples_all_shaping, dim =1))

In [ ]:
E_IS_SCOPE

tensor(-1.1470e+24, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
E_IS_SCOPE_2

tensor(-1.5943e+26, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
samples_IS_SCOPE[0]

tensor([ -2.5405e-85,  -0.0000e+00,   7.9024e-62,  -2.9790e-61,  -3.1831e-75,
         -1.5365e-64,  -0.0000e+00,  -8.3521e-71,   9.1314e-24,  -6.9648e-66,
        -2.0605e-135,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,
         -1.5136e-54,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,  -0.0000e+00,
          0.0000e+00,   7.9024e-62,   0.0000e+00,  -0.0000e+00,  2.1164e-111,
        -9.5684e-135,  -0.0000e+00, -1.4140e-116,  -3.3219e-82,  -0.0000e+00,
         -1.5197e-87,  -2.5695e-67,   5.3077e-82,  -0.0000e+00,   2.3741e-47,
         -0.0000e+00,  -1.8253e-71,  -3.4879e+11,  -0.0000e+00,  -0.0000e+00,
         3.8146e-114,  -0.0000e+00,  -7.5975e-31,  1.2321e-124,  -0.0000e+00,
         -0.0000e+00, -2.0958e-170,   0.0000e+00,  -0.0000e+00,  -0.0000e+00,
         -0.0000e+00, -1.0845e-110,  -1.8398e-69,  -1.8253e-71,   0.0000e+00,
         -1.0142e-82,  -3.8087e-71, -4.7846e-116,  -0.0000e+00,  -1.1575e+14,
         -8.3983e-61,  -0.0000e+00,  -3.5652e-20,  -4.5806e-83, 

In [ ]:
sample_sums_states_weight_diff[0]

tensor([ 5.1683e-01,  5.2720e-01,  2.3982e+00,  5.2951e-01,  5.1357e-01,
         5.1364e-01, -1.1535e+00,  5.1631e-01,  5.1753e-01,  5.1325e-01,
         9.0952e-02,  3.6275e-01,  5.2720e-01,  7.6119e-02,  7.6752e-02,
         5.3759e-01,  9.0952e-02,  7.6337e-02,  9.0952e-02,  9.0657e-02,
         5.7605e+00,  2.3982e+00,  3.6205e+00,  9.0436e-02,  3.9866e+00,
         7.6361e-02,  5.1632e-01,  7.7653e-02,  1.2674e-01,  9.0436e-02,
         7.6119e-02,  5.1709e-01,  3.6920e+00,  5.1663e-01,  9.0973e-02,
         5.3759e-01,  5.3762e-01, -2.9977e+06,  5.1357e-01,  5.3602e-01,
         4.1946e+00,  5.2720e-01, -6.4696e+01,  5.2645e-01,  5.1683e-01,
         5.2907e-01,  5.2645e-01,  3.6920e+00,  5.3771e-01,  4.8009e-01,
         4.2201e-01,  9.0952e-02,  8.3283e-02,  5.3762e-01,  4.2676e+00,
         5.1366e-01,  7.7653e-02,  5.3583e-01,  5.3728e-01, -5.6362e+07,
         2.5242e-01, -1.5415e+10, -1.8276e-02,  8.9459e-02,  9.0952e-02,
         5.1665e-01, -2.1687e+00, -4.6237e+01,  5.2

In [ ]:
sums_shaping = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

In [ ]:
torch.mean(sums_shaping)

tensor(-9.7904e+39, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [ ]:
# Print the min and max values of tensors
print("padded_weight_diff_tensors - Min:", padded_weight_diff_tensors.min().item(), " Max:", padded_weight_diff_tensors.max().item())
print("sums_states_weight_diff - Min:", sums_states_weight_diff.min().item(), " Max:", sums_states_weight_diff.max().item())
print("gamma_weights_last_tensor - Min:", gamma_weights_last_tensor.min().item(), " Max:", gamma_weights_last_tensor.max().item())
print("samples_IS - Min:", samples_IS.min().item(), " Max:", samples_IS.max().item())


padded_weight_diff_tensors - Min: -6979572071212940.0  Max: 1.839130399958296e+16
sums_states_weight_diff - Min: -5225342891061462.0  Max: 1365487.6953395614
gamma_weights_last_tensor - Min: 1.846463839642936e-238  Max: 143894458319084.28
samples_IS - Min: -10076.89890923087  Max: 159882731465649.2


In [ ]:
torch.mean(sample_sums_states_weight_diff,dim=1)

tensor([-9.9641e+39, -2.6692e+27, -9.9641e+39,  ..., -1.9928e+40,
        -2.9892e+40, -1.3346e+27], dtype=torch.float64,
       grad_fn=<MeanBackward1>)

In [ ]:
torch.mean(samples_gamma_weight_states_last_sub_states_first, dim =1)

tensor([1.7368e+38, 5.5852e+13, 1.7368e+38,  ..., 3.4735e+38, 5.2103e+38,
        2.7925e+13], dtype=torch.float64, grad_fn=<MeanBackward1>)

# Calc Variance

In [ ]:
def calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE):

  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(samples_IS, dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(samples_IS, dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(sample_sums_states_weight_diff, dim = 1))**2

  # all terms
  # SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first
  # E_IS_SCOPE = torch.mean(torch.mean(samples_IS, dim =1) * torch.mean(SCOPE, dim =1))
  # E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  E_IS_SCOPE = torch.mean(torch.mean(samples_IS_SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(samples_IS,dim = 1 )) * torch.mean(torch.mean(samples_all_shaping, dim =1))



  # SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE
  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_s_wdiff_all_sq

  IS_variance = E_IS_sq - E_IS_all_sq

  return E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, IS_variance, SCOPE_variance


In [ ]:
SCOPE = sample_sums_states_weight_diff-samples_gamma_weight_states_last_sub_states_first


In [ ]:
_,_,_,_,_,_,IS_variance, SCOPE_variance = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE)

In [ ]:
print(f'IS Variance: {IS_variance.item()} SCOPE Variance: {SCOPE_variance.item()}')

IS Variance: 2.514807940970177e+22 SCOPE Variance: 5.892057226414652e+25


In [ ]:
SCOPE_variance

tensor([2.0262e+82, 1.7582e+80, 2.0262e+82,  ..., 4.0349e+82, 6.0435e+82,
        1.7582e+80], dtype=torch.float64, grad_fn=<SubBackward0>)

# Optimizing

In [ ]:
import torch.optim as optim

In [ ]:
import torch
import torch.optim as optim

def train_var(model, num_epochs, learning_rate, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1):
    model.train()

    # Enable anomaly detection
    torch.autograd.set_detect_anomaly(True)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0

        # Forward pass
        states_output, states_first_output, states_last_output = test1.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
        sums_states_weight_diff = test1.states_weight_diff_sums(states_output, padded_weight_diff_tensors)
        gamma_weights_states_last_sub_states_first = test1.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)
        # sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)

        samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE = test1.bootstrap_all_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor)


        E_IS_sq, E_IS_all_sq, E_s_wdiff_sq, E_s_wdiff_all_sq, E_IS_SCOPE, E_IS_E_SCOPE, _, variance_loss = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, samples_all_shaping, samples_IS_SCOPE)

        print(f"Epoch {epoch+1}")
        print("Var loss: ", variance_loss)

        # Print each term
        print(f"E_IS_sq: {E_IS_sq}")
        print(f"E_IS_all_sq: {E_IS_all_sq}")
        print(f"E_s_wdiff_sq: {E_s_wdiff_sq}")
        print(f"E_s_wdiff_all_sq: {E_s_wdiff_all_sq}")
        print(f"E_IS_SCOPE: {E_IS_SCOPE}")
        print(f"E_IS_E_SCOPE: {E_IS_E_SCOPE}")

        tot = variance_loss

        optimizer.zero_grad()

        # Retain the graph to avoid clearing it before backward pass
        tot.backward(retain_graph=True)

        optimizer.step()

        total_loss += tot.item()

        print(f"Total Loss: {total_loss}")
        print("-" * 40)

    # Disable anomaly detection after running the code
    torch.autograd.set_detect_anomaly(False)

    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"Parameter name: {name}")
            print(f"Weights: {param.data}")

    return model


In [ ]:
model2 = train_var(model, 20, 0.0001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(-3.0838e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 5.210967549166126e+22
E_s_wdiff_all_sq: 2.5172984363039316e+22
E_IS_SCOPE: -1.5459488896936224e+25
E_IS_E_SCOPE: -1.4417566494849713e+22
Total Loss: -3.0838057890344423e+25
----------------------------------------
Epoch 2
Var loss:  tensor(-3.3585e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 6.546944000346388e+22
E_s_wdiff_all_sq: 3.517525168586692e+22
E_IS_SCOPE: -1.6839284068856547e+25
E_IS_E_SCOPE: -1.8983210591791954e+22
Total Loss: -3.358515944880221e+25
----------------------------------------
Epoch 3
Var loss:  tensor(-3.6329e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 8.127328468140865e+22
E_s_wdiff_all_sq: 4.669789629828819e+22
E_IS_SCOPE: -1.82179

In [ ]:
model5 = train_var(model4, 500, 0.0001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(1.6838e+21, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.2080531216272573e+22
E_s_wdiff_all_sq: 7.854446300022197e+18
E_IS_SCOPE: -3.2924307594909997e+22
E_IS_E_SCOPE: -1.0101281026055919e+22
Total Loss: 1.683838514322106e+21
----------------------------------------
Epoch 2
Var loss:  tensor(2.4018e+21, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.5681370975049804e+22
E_s_wdiff_all_sq: 5.3651102132735515e+20
E_IS_SCOPE: -2.958920317364135e+22
E_IS_E_SCOPE: -6.916923741107582e+21
Total Loss: 2.401815260791626e+21
----------------------------------------
Epoch 3
Var loss:  tensor(2.3126e+21, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.060980670643068e+22
E_s_wdiff_all_sq: 2.9444178650483524e+20
E_IS_SCOPE: -3.6706298454

KeyboardInterrupt: 

In [ ]:
model2 = train_var(model, 200, 0.0001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(-5.0872e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.2091641687126665e+22
E_s_wdiff_all_sq: 8.421209012866424e+18
E_IS_SCOPE: -2.547315218031649e+25
E_IS_E_SCOPE: -1.2421832478324517e+22
Total Loss: -5.087179914210118e+25
----------------------------------------
Epoch 2
Var loss:  tensor(-5.3797e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.1588018418409055e+22
E_s_wdiff_all_sq: 8.04536804186359e+20
E_IS_SCOPE: -2.694291367756086e+25
E_IS_E_SCOPE: -1.7349946687008094e+22
Total Loss: -5.379704141723259e+25
----------------------------------------
Epoch 3
Var loss:  tensor(-5.6711e+25, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 1.4111873352395464e+22
E_s_wdiff_all_sq: 3.463374565893483e+21
E_IS_SCOPE: -2.8408122

KeyboardInterrupt: 

In [ ]:
model3 = train_var(model, 20, 0.001, samples_IS, padded_state_tensors, states_first_tensor, states_last_tensor, samples_all_shaping, samples_IS_SCOPE, test1)

Epoch 1
Var loss:  tensor(-1.3998e+26, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 2.7000520983736854e+26
E_s_wdiff_all_sq: 1.1991158006418987e+26
E_IS_SCOPE: -2.0238984353616566e+26
E_IS_E_SCOPE: 1.7432066784634587e+24
Total Loss: -1.3997894919094345e+26
----------------------------------------
Epoch 2
Var loss:  tensor(-1.8930e+26, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 2.4851380256092592e+26
E_s_wdiff_all_sq: 1.0868796875981285e+26
E_IS_SCOPE: -2.1642866384407017e+26
E_IS_E_SCOPE: 1.6592145943890643e+24
Total Loss: -1.892960208309719e+26
----------------------------------------
Epoch 3
Var loss:  tensor(-2.3750e+26, dtype=torch.float64, grad_fn=<SubBackward0>)
E_IS_sq: 5.058576277992128e+22
E_IS_all_sq: 2.543768337021951e+22
E_s_wdiff_sq: 2.2864222596610805e+26
E_s_wdiff_all_sq: 9.831123864853387e+25
E_IS_SCOPE: -2.307

# Working on prep

In [ ]:
# # Working on padding trajectories to allow for easier optimization

# def pad_trajectories(pi_b):
#     # Find the maximum length among all trajectories
#     max_length = max(len(traj) for traj in pi_b)

#     # Define the padding value
#     padding_value = np.array([np.array([0, 0]), 0, 0, np.array([0, 0]), 0, 0], dtype=object)

#     # Pad each trajectory to match the maximum length
#     padded_pi_b = [traj + [padding_value] * (max_length - len(traj)) for traj in pi_b]

#     return padded_pi_b

In [ ]:
# pi_b_padded = pad_trajectories(pi_b)

In [ ]:
def variance_terms_tens(eval_policy, behav_policy, behavior_policies):
  # Initialize lists to store axis data for each policy
  timesteps = []
  states = []
  state_first = []
  state_last = []
  actions = []
  rewards = []
  gamma_last = []
  weight_last = []
  weights = calculate_importance_weights(eval_policy, behav_policy, behavior_policies)
  psi = []

  for index, policy in enumerate(behavior_policies):
      policy_array = np.array(policy)
      timesteps.append(policy_array[:, 4].astype(int))
      # s.append(policy_array[:, 0])

      # last timestep for gamma
      gamma_last.append(len(policy))
      # last importance weight
      weight_last.append(weights[index][-1])


      states.append(policy_array[:, 0][1:])
      psi.append(policy_array[:,5][1:])
      state_first.append(policy_array[:,0][0])
      state_last.append(policy_array[:,0][-1])
      actions.append(policy_array[:, 1])
      rewards.append(policy_array[:, 2].astype(float))

  weights_difference = []
  for index, weight in enumerate(weights):
    # diff = np.array(w[index][:-1]) - np.array(w[index][1:])
    diff = np.array(weight[:-1]) - np.array(weight[1:])
    weights_difference.append(diff)

  return timesteps, states, state_first, state_last, actions, rewards, gamma_last, weight_last, weights, weights_difference

In [ ]:
# t, s, s_f, s_l, a, r, g_l, w_l, w, w_diff = variance_terms_tens(P_pi_e, P_pi_b, pi_b)

In [ ]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = variance_terms_tens(P_pi_e, P_pi_b, pi_b)

In [ ]:
import numpy as np

def padding_IS_terms(timesteps, actions, rewards, weights):
    # Find the maximum length among all lists
    max_length = max(len(traj) for traj in timesteps)

    # Define the padding values
    zero_padding = 0

    # Pad each list to match the maximum length
    padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
    padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
    padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
    padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

    return padded_timesteps, padded_rewards, padded_actions, padded_weights

In [ ]:
padded_timesteps, padded_rewards, padded_actions, padded_weights = padding_IS_terms(timesteps, actions, rewards, weights)

In [ ]:
def padding_states_weights_difference(states, weights_difference):
    # Find the maximum length of trajectories
    max_length = max(len(trajectory) for trajectory in states)

    zero_padding = 0

    # Pad each trajectory to make them all the same length
    padded_states = [
        [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
        for trajectory in states
    ]

    padded_weights_difference = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights_difference]

    return padded_states, padded_weights_difference

In [ ]:
padded_states, padded_weights_difference = padding_states_weights_difference(states, weights_difference)

In [ ]:
def tensorize_padded_terms(padded_states, padded_weights_difference):
  padded_state_tensors = torch.tensor(padded_states, dtype = torch.float64)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float64)
  padded_weight_diff_tensors = padded_weight_diff_tensors.unsqueeze(-1)

  return padded_state_tensors, padded_weight_diff_tensors


In [ ]:
padded_state_tensors, padded_weight_diff_tensors = tensorize_padded_terms(padded_states, padded_weights_difference)

<ipython-input-31-3c22136e8020>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = torch.float64)


In [ ]:
def tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last):
  states_first_tensor = torch.tensor(states_first, dtype = torch.float64)
  states_last_tensor = torch.tensor(states_last, dtype = torch.float64)
  gamma_last_tensor = torch.tensor(gamma_last, dtype = torch.float64)
  weights_last_tensor = torch.tensor(weights_last, dtype = torch.float64)

  return states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor

In [ ]:
states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)

In [ ]:
def calc_IS_terms(gamma, timesteps, rewards, weights):
  gtrw = np.power(gamma, timesteps)*rewards*weights

  IS_tensor = torch.sum(torch.tensor(gtrw, dtype = torch.float32), dim = 1, keepdim = True)

  return IS_tensor


In [ ]:
IS_tensor = calc_IS_terms(0.9, padded_timesteps, padded_rewards, padded_weights)

In [ ]:
torch.finfo(torch.float64).max

1.7976931348623157e+308

In [ ]:
# samples_IS[0]

In [ ]:
def calc_gamma_weight_last(gamma, gamma_last, weights_last):
  gamma_weight_last = np.power(gamma, gamma_last)*weights_last

  gamma_weight_last_tensor = torch.tensor(gamma_weight_last, dtype = torch.float64).unsqueeze(-1)

  return gamma_weight_last_tensor

In [ ]:
gamma_weights_last_tensor = calc_gamma_weight_last(0.9, gamma_last, weights_last)

In [ ]:
def bootstrap_IS_terms(IS_tensor, num_samples):
  seed = 42
  torch.manual_seed(seed)

  num_bootstraps = num_samples*len(IS_tensor)

  # Sample indices with replacement
  sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

  # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
  new_size = (num_samples, IS_tensor.shape[0])

  IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

  # sampled_tensor = IS_bootstraps.view(new_size)

  return IS_bootstraps

In [ ]:
samples_IS = bootstrap_IS_terms(IS_tensor, 10000)

In [ ]:
def clamp_large_terms(input_term):
  max_float64 = torch.finfo(torch.float64).max
  return torch.clamp(input_term, min=-max_float64, max=max_float64)


In [ ]:
def clamp_large_terms(input_term):
    max_float64 = torch.tensor(torch.finfo(torch.float64).max, dtype=torch.float64)
    min_value = torch.tensor(-1e50, dtype=torch.float64)
    return torch.clamp(input_term, min=min_value, max=max_float64)

    # return torch.clamp(input_term, min=-max_float64, max=max_float64)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CustomizableFeatureNet_d(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout_prob=0.2, dtype=torch.float32):
        super(CustomizableFeatureNet_d, self).__init__()
        self.hidden_layers = nn.ModuleList()

        # Create the hidden layers based on the provided sizes
        for in_dim, out_dim in zip([input_dim] + hidden_dims, hidden_dims):
            self.hidden_layers.append(nn.Linear(in_dim, out_dim).to(dtype))

        self.output_layer = nn.Linear(hidden_dims[-1], output_dim).to(dtype)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

In [ ]:
model = CustomizableFeatureNet_d(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [ ]:
# model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
def pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor):
  # Get model outputs for states
  states_output = model(padded_state_tensors)
  states_first_output = model(states_first_tensor)
  states_last_output = model(states_last_tensor)
  return states_output, states_first_output, states_last_output

In [ ]:
states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)

In [ ]:
def states_weight_diff_sums(states_output, padded_weight_diff_tensors):
  states_weight_diff = states_output * padded_weight_diff_tensors
  sums_states_weight_diff = torch.sum(states_weight_diff, dim =1)

  return sums_states_weight_diff

In [ ]:
sums_states_weight_diff = states_weight_diff_sums(states_output, padded_weight_diff_tensors)

In [ ]:
def last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output):
  gamma_weights_states_last_sub_states_first = gamma_weights_last_tensor*states_last_output -  states_first_output

  return gamma_weights_states_last_sub_states_first

In [ ]:
gamma_weights_states_last_sub_states_first = last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)

In [ ]:
# gamma_weights_states_last_sub_states_first.shape

In [ ]:
def bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, num_samples):

  seed = 42
  torch.manual_seed(seed)

  num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

  # Sample indices with replacement
  sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)

  # sizes
  # size_states_weights_diff = (num_samples, states_output.shape[0], states_output.shape[1])
  reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

  # Resize samples to shape num_samples x num_trajectories x length_padded_trajectories
  # samples_states_output = states_output[sampled_indices].view(size_states_weights_diff)
  # samples_weight_diff = padded_weight_diff_tensors[sampled_indices].view(size_states_weights_diff)

  # Resize samples to shape num_samples x num_trajectories
  sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
  # samples_states_first_output = sums_states_weight_diff[sampled_indices].view(reshaped_size)
  # samples_states_last_output = states_last_output[sampled_indices].view(reshaped_size)
  samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)


  return sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first


In [ ]:
sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first = bootstrap_shaping_terms(sums_states_weight_diff, gamma_weights_states_last_sub_states_first, 10000)

In [ ]:
# def gtrw_plus_states_weight_diff_min_last_firsts_terms(gtrw_tensor, sums_states_weight_diff, gamma_weights_states_last_sub_states_first):
#   sum_IS_and_shaping = gtrw_tensor + sums_states_weight_diff + gamma_weights_states_last_sub_states_first

#   return sum_IS_and_shaping

In [ ]:
# def bootstrap_IS_shaping_terms(gtrw_tensor, sums_states_weight_diff, gamma_weights_states_last_sub_states_first):
#   seed = 42
#   torch.manual_seed(seed)

#   num_bootstraps = num_samples*sums_states_weight_diff.shape[0]

#   # Sample indices with replacement
#   sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps,), dtype=torch.long)



In [ ]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim = 1))))

In [ ]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim =1)**2)))

In [ ]:
# clamp_large_terms(torch.mean(clamp_large_terms(torch.mean(samples_IS, dim =1))**2) - torch.mean(clamp_large_terms(torch.mean(samples_IS, dim = 1))))

In [ ]:
# SCOPE = sample_sums_states_weight_diff+samples_gamma_weight_states_last_sub_states_first

In [ ]:
# torch.mean(SCOPE,dim =1).shape

In [ ]:
torch.mean(clamp_large_terms(samples_IS), dim = 1)

tensor([      -inf, 2.0056e+14,       -inf,  ...,       -inf,       -inf,
        1.0028e+14])

In [ ]:
def calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first):


  # states_output, states_first_output, states_last_output = pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)



  # Begin calcs without clamping

  # IS
  E_IS_sq = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1)**2)
  E_IS_all_sq = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1))**2

  # states_weight_diff
  E_s_wdiff_sq = torch.mean(torch.mean(clamp_large_terms(sample_sums_states_weight_diff), dim =1)**2)
  E_s_wdiff_all_sq = torch.mean(torch.mean(clamp_large_terms(sample_sums_states_weight_diff), dim = 1))**2

  # all terms
  SCOPE = clamp_large_terms(sample_sums_states_weight_diff)+clamp_large_terms(samples_gamma_weight_states_last_sub_states_first)
  E_IS_SCOPE = torch.mean(torch.mean(clamp_large_terms(samples_IS), dim =1) * torch.mean(SCOPE, dim =1))
  E_IS_E_SCOPE = torch.mean(torch.mean(clamp_large_terms(samples_IS),dim = 1 ))*torch.mean(torch.mean(SCOPE, dim =1))

  SCOPE_variance = E_IS_sq + 2*E_IS_SCOPE + E_s_wdiff_sq - E_IS_all_sq - 2*E_IS_E_SCOPE - E_IS_E_SCOPE

  IS_variance = E_IS_sq - E_IS_all_sq

  return IS_variance, SCOPE_variance


In [ ]:
IS_variance, SCOPE_variance = calculate_shaped_variance(samples_IS, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first)

In [ ]:
torch.mean(torch.mean(clamp_large_terms(samples_IS), dim = 1)**2)

tensor(inf)

In [ ]:
SCOPE_variance

tensor(nan, dtype=torch.float64, grad_fn=<SubBackward0>)

In [ ]:
IS_variance

tensor(nan)

In [ ]:
import torch

def clamp_large_terms(input_term):
    max_float64 = torch.tensor(torch.finfo(torch.float64).max, dtype=torch.float64)
    # min_value = torch.tensor(-1e38, dtype=torch.float64)
    min_value = torch.tensor(-9e-38, dtype=torch.float64)

    # min_value = torch.tensor(-1.7e+308, dtype = torch.float64)
    # min_value = torch.tensor(torch.finfo(torch.float64).min, dtype=torch.float64)


    # Using torch.where to explicitly set values outside the desired range
    clamped_result = torch.where(input_term < min_value, min_value, input_term)
    clamped_result = torch.where(clamped_result > max_float64, max_float64, clamped_result)

    return clamped_result
clamp_large_terms(samples_IS)[0].min()

tensor(-9.0000e-38)

In [ ]:
samples_IS[0]

tensor([ 0.0000e+00, -9.5567e-34, -1.1020e-12, -5.7703e-20,  0.0000e+00,
        -6.6609e-06,  0.0000e+00,  0.0000e+00,  0.0000e+00, -7.4562e-17,
        -1.3518e-33, -6.3068e-29, -1.1566e-08,  0.0000e+00, -3.2522e-24,
         0.0000e+00,  0.0000e+00, -4.0924e-02,  0.0000e+00,  0.0000e+00,
        -6.0833e-21, -1.1020e-12,  0.0000e+00, -3.5032e-43, -3.4013e-37,
        -7.9832e-23,  0.0000e+00, -2.2156e-12,  0.0000e+00, -7.3671e-27,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00, -3.0953e-12,  0.0000e+00, -7.1745e-11,  0.0000e+00,
        -1.2184e+02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -4.8370e-19,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00, -1.7865e-25, -3.0953e-12, -3.1972e-14,
         0.0000e+00, -3.9939e-21,  0.0000e+00, -4.2383e-17,  0.0000e+00,
         0.0000e+00, -9.5567e-34,  0.0000e+00,  0.0000e+00, -5.4262e-07,
        -3.7487e-37, -2.6165e-41,  0.0000e+00, -1.0

In [ ]:
clamp_large_terms(samples_IS)[0].min()

tensor(-9.0000e-38)

In [ ]:
def clamp_large_terms(tensor):
    max_float64 = torch.finfo(torch.float64).max
    min_float64 = torch.finfo(torch.float64).min

    tensor[tensor == float('inf')] = 1e38  # Choose a large finite value
    tensor[tensor == float('-inf')] = -1e38  # Choose a small finite value

    return tensor

# Example usage:
# samples_IS_clamped = clamp_tensor(samples_IS)

In [ ]:
# samples_IS_clamped[0]

NameError: name 'samples_IS_clamped' is not defined

In [ ]:
torch.tensor(torch.finfo(torch.float64).min, dtype=torch.float64)

tensor(-1.7977e+308, dtype=torch.float64)

In [ ]:
model.eval()

CustomizableFeatureNet_d(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=32, bias=True)
  )
  (output_layer): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
output = model(state_tensors)

RuntimeError: mat1 and mat2 must have the same dtype, but got Float and Double

In [ ]:
output.shape

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
scope_set, phi_set = subset_policies(pi_b, 0.3)

In [ ]:
IS, state_tensors, w_diff, f, st_og, psi_res, sample_last_tensors = variance_terms_tens(P_pi_e, P_pi_b, phi_set)

In [ ]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1)

In [ ]:
model2 = train_mse_var(model, 40, 0.001, 1, 1, IS, state_tensors, w_diff, f, sample_last_tensors, phi_set, st_og, psi_res)

# Test class

In [22]:
# import numpy as np
# from IS import calculate_importance_weights

# import torch

class SCOPE_variance(object):
    def __init__(self, model, gamma, num_bootstraps, pi_b, P_pi_b, P_pi_e, dtype):
        self.model = model
        self.gamma = gamma
        self.num_bootstraps = num_bootstraps
        self.pi_b = pi_b
        self.P_pi_b = P_pi_b
        self.P_pi_e = P_pi_e
        self.dtype = dtype


    def prep_policies(self):
        # Initialize lists to store axis data for each policy
        timesteps = []
        states = []
        state_first = []
        state_last = []
        actions = []
        rewards = []
        gamma_last = []
        weight_last = []
        weights = calculate_importance_weights(self.P_pi_e, self.P_pi_b, self.pi_b)
        psi = []

        for index, policy in enumerate(self.pi_b):
            policy_array = np.array(policy)
            timesteps.append(policy_array['timestep'].astype(int))
            # s.append(policy_array[:, 0])

            # last timestep for gamma
            gamma_last.append(len(policy))
            # last importance weight
            weight_last.append(weights[index][-1])


            states.append(policy_array['state'][1:])
            psi.append(policy_array['psi'][1:])
            state_first.append(policy_array['state'][0])
            state_last.append(policy_array['state'][-1])
            actions.append(policy_array['action'])
            rewards.append(policy_array['reward'].astype(float))

        weights_difference = []
        for index, weight in enumerate(weights):
            # diff = np.array(w[index][:-1]) - np.array(w[index][1:])
            diff = np.array(weight[:-1]) - np.array(weight[1:])
            weights_difference.append(diff)

        return timesteps, states, state_first, state_last, actions, rewards, gamma_last, weight_last, weights, weights_difference

    def padding_IS_terms(self,timesteps, actions, rewards, weights):
        # Find the maximum length among all lists
        max_length = max(len(traj) for traj in timesteps)

        # Define the padding values
        zero_padding = 0

        # Pad each list to match the maximum length
        padded_timesteps = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in timesteps]
        padded_rewards = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in rewards]
        padded_actions = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in actions]
        padded_weights = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights]

        return padded_timesteps, padded_rewards, padded_actions, padded_weights

    def padding_states_weights_difference(self, states, weights_difference):
        # Find the maximum length of trajectories
        max_length = max(len(trajectory) for trajectory in states)

        zero_padding = 0

        # Pad each trajectory to make them all the same length
        padded_states = [
            [list(item) for item in trajectory] + [[0, 0]] * (max_length - len(trajectory))
            for trajectory in states
        ]

        padded_weights_difference = [np.concatenate([traj, [zero_padding] * (max_length - len(traj))]) for traj in weights_difference]

        return padded_states, padded_weights_difference

    def tensorize_padded_terms(self, padded_states, padded_weights_difference):
        padded_state_tensors = torch.tensor(padded_states, dtype = self.dtype)
        padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)
        padded_weight_diff_tensors = padded_weight_diff_tensors.unsqueeze(-1)

        return padded_state_tensors, padded_weight_diff_tensors

    def tensorize_last_and_first_terms(self, states_first, states_last, gamma_last, weights_last):
        states_first_tensor = torch.tensor(states_first, dtype = self.dtype)
        states_last_tensor = torch.tensor(states_last, dtype = self.dtype)
        gamma_last_tensor = torch.tensor(gamma_last, dtype = self.dtype)
        weights_last_tensor = torch.tensor(weights_last, dtype = self.dtype)

        return states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor

    def calc_IS_terms(self, gamma, timesteps, rewards, weights):
        gtrw = np.power(gamma, timesteps)*rewards*weights

        IS_tensor = torch.sum(torch.tensor(gtrw, dtype = self.dtype), dim = 1, keepdim = True)

        return IS_tensor

    def calc_gamma_weight_last(self, gamma, gamma_last, weights_last):
        gamma_weight_last = np.power(gamma, gamma_last)*weights_last

        gamma_weight_last_tensor = torch.tensor(gamma_weight_last, dtype = self.dtype).unsqueeze(-1)

        return gamma_weight_last_tensor
    def bootstrap_IS_terms(self, IS_tensor, num_samples):
        seed = 42
        torch.manual_seed(seed)

        num_bootstraps = num_samples*len(IS_tensor)

        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

        # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
        new_size = (num_samples, IS_tensor.shape[0])

        IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

        # sampled_tensor = IS_bootstraps.view(new_size)

        return IS_bootstraps

    def states_weight_diff_sums(self, states_output, padded_weight_diff_tensors):
        states_weight_diff = states_output * padded_weight_diff_tensors
        sums_states_weight_diff = torch.sum(states_weight_diff, dim =1)

        return sums_states_weight_diff

    def last_first_terms_operations(self, gamma_weights_last_tensor, states_last_output, states_first_output):
        gamma_weights_states_last_sub_states_first = gamma_weights_last_tensor*states_last_output -  states_first_output

        return gamma_weights_states_last_sub_states_first

    def bootstrap_shaping_terms(self, sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor):

        seed = 42
        torch.manual_seed(seed)

        num_samples = self.num_bootstraps

        num_bootstraps_lin = num_samples*sums_states_weight_diff.shape[0]

        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps_lin,), dtype=torch.long)

        reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

        # Resize samples to shape num_samples x num_trajectories
        sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
        samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)

        # Sum states_weight_diff and gamma_weights-states_last_sub_states_first
        sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

        # sample IS terms

        IS_SCOPE = IS_tensor * sum_terms

        samples_IS_SCOPE = IS_SCOPE[sampled_indices].view(reshaped_size)


        sample_all_shaping = sum_terms[sampled_indices].view(reshaped_size)

        return sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, sample_all_shaping, samples_IS_SCOPE

    def bootstrap_all_terms(self, sums_states_weight_diff, gamma_weights_states_last_sub_states_first, IS_tensor):
        seed = 42
        torch.manual_seed(seed)

        # num_bootstraps = num_samples*len(IS_tensor)

        # Sample indices with replacement
        # sampled_indices = torch.randint(0, len(IS_tensor), size=(num_bootstraps,), dtype=torch.long)

        # new_size = (num_samples, IS_tensor.shape[0], IS_tensor.shape[1])
        # new_size = (num_samples, IS_tensor.shape[0])

        # IS_bootstraps = IS_tensor[sampled_indices].view(new_size)

        num_samples = self.num_bootstraps
        num_bootstraps_lin = num_samples*sums_states_weight_diff.shape[0]


        # Sample indices with replacement
        sampled_indices = torch.randint(0, len(sums_states_weight_diff), size=(num_bootstraps_lin,), dtype=torch.long)

        reshaped_size = (num_samples, sums_states_weight_diff.shape[0])

        IS_bootstraps = IS_tensor[sampled_indices].view(reshaped_size)

        # Resize samples to shape num_samples x num_trajectories
        sample_sums_states_weight_diff = sums_states_weight_diff[sampled_indices].view(reshaped_size)
        samples_gamma_weight_states_last_sub_states_first = gamma_weights_states_last_sub_states_first[sampled_indices].view(reshaped_size)

        # Sum states_weight_diff and gamma_weights-states_last_sub_states_first
        sum_terms = sums_states_weight_diff + gamma_weights_states_last_sub_states_first

        # sample IS terms

        IS_SCOPE = IS_tensor * sum_terms

        samples_IS_SCOPE = IS_SCOPE[sampled_indices].view(reshaped_size)

        sample_all_shaping = sum_terms[sampled_indices].view(reshaped_size)

        return IS_bootstraps, sample_sums_states_weight_diff, samples_gamma_weight_states_last_sub_states_first, sample_all_shaping, samples_IS_SCOPE



    def pass_states(self,model, padded_state_tensors, states_first_tensor, states_last_tensor):
        # Get model outputs for states
        states_output = model(padded_state_tensors)
        states_first_output = model(states_first_tensor)
        states_last_output = model(states_last_tensor)
        return states_output, states_first_output, states_last_output


    # def prepare(self):
    #     timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = self.prep_policies()
    #     padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
    #     padded_states, padded_weights_difference = self.padding_states_weights_difference(states, weights_difference)
    #     padded_state_tensors, padded_weight_diff_tensors = self.tensorize_padded_terms(padded_states, padded_weights_difference)
    #     states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = self.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
    #     IS_tensor = self.calc_IS_terms(self.gamma, padded_timesteps, padded_rewards, padded_weights)
    #     gamma_weights_last_tensor = self.calc_gamma_weight_last(self.gamma, gamma_last, weights_last)
    #     samples_IS = self.bootstrap_IS_terms(IS_tensor, self.num_bootstraps)

    #     return IS_tensor, samples_IS, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor


    def prepare(self):
        timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = self.prep_policies()
        padded_timesteps, padded_rewards, padded_actions, padded_weights = self.padding_IS_terms(timesteps, actions, rewards, weights)
        padded_states, padded_weights_difference = self.padding_states_weights_difference(states, weights_difference)
        padded_state_tensors, padded_weight_diff_tensors = self.tensorize_padded_terms(padded_states, padded_weights_difference)
        states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = self.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
        IS_tensor = self.calc_IS_terms(self.gamma, padded_timesteps, padded_rewards, padded_weights)
        gamma_weights_last_tensor = self.calc_gamma_weight_last(self.gamma, gamma_last, weights_last)
        # samples_IS = self.bootstrap_IS_terms(IS_tensor, self.num_bootstraps)

        return IS_tensor, padded_state_tensors, padded_weight_diff_tensors, gamma_weights_last_tensor, states_first_tensor, states_last_tensor

In [77]:
pi_b

[array([([2., 9.], 3, 0., [1., 9.],  0, 3.),
        ([1., 9.], 3, 0., [0., 9.],  1, 4.),
        ([0., 9.], 1, 0., [0., 8.],  2, 5.),
        ([0., 8.], 1, 0., [0., 7.],  3, 5.),
        ([0., 7.], 1, 0., [0., 6.],  4, 5.),
        ([0., 6.], 1, 0., [0., 5.],  5, 5.),
        ([0., 5.], 1, 0., [0., 4.],  6, 5.),
        ([0., 4.], 4, 0., [1., 4.],  7, 6.),
        ([1., 4.], 1, 0., [1., 3.],  8, 5.),
        ([1., 3.], 4, 0., [2., 3.],  9, 6.),
        ([2., 3.], 4, 0., [3., 3.], 10, 5.),
        ([3., 3.], 4, 0., [4., 3.], 11, 4.),
        ([4., 3.], 1, 0., [4., 2.], 12, 3.),
        ([4., 2.], 1, 0., [5., 2.], 13, 4.),
        ([5., 2.], 4, 0., [6., 2.], 14, 3.),
        ([6., 2.], 1, 0., [6., 1.], 15, 3.),
        ([6., 1.], 1, 0., [7., 1.], 16, 4.),
        ([7., 1.], 4, 0., [8., 1.], 17, 4.),
        ([8., 1.], 0, 0., [8., 1.], 18, 4.),
        ([8., 1.], 3, 0., [7., 1.], 19, 4.),
        ([7., 1.], 4, 0., [8., 1.], 20, 4.),
        ([8., 1.], 3, 0., [7., 1.], 21, 4.),
        ([

In [82]:
pi_b[states[0],actions[0]]

array([3, 3, 1, 1, 1, 1, 1, 4, 1, 4, 4, 4, 1, 1, 4, 1, 1, 4, 0, 3, 4, 3,
       1])

# Class play

In [23]:
P_pi_b = action_probs_top_n_epsilon(q_table, 1, 0.4)
pi_b = experiment_actions(5, env, P_pi_b)
P_pi_e = action_probs_top_n_epsilon(q_table, 2, 0.05)
pi_e = experiment_actions(5, env, P_pi_e)

In [25]:
model = CustomizableFeatureNet(input_dim=2, hidden_dims=[16, 32], output_dim=1, dtype = torch.float64)

In [26]:
testing = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e, dtype = torch.float64)

In [30]:
timesteps, states, states_first, states_last, actions, rewards, gamma_last, weights_last, weights, weights_difference = testing.prep_policies()


In [32]:
P_pi_b[tuple(np.append(pi_b[0]['state'][0].astype(int) , (pi_b[0]['action'][0],)))]

0.6799999999999999

In [66]:
P_pi_e[tuple(np.append(pi_b[0]['state'][0].astype(int) , (pi_b[0]['action'][0],)))]

0.485

In [76]:
pi_b[0]

array([([2., 9.], 3, 0., [1., 9.],  0, 3.),
       ([1., 9.], 3, 0., [0., 9.],  1, 4.),
       ([0., 9.], 1, 0., [0., 8.],  2, 5.),
       ([0., 8.], 1, 0., [0., 7.],  3, 5.),
       ([0., 7.], 1, 0., [0., 6.],  4, 5.),
       ([0., 6.], 1, 0., [0., 5.],  5, 5.),
       ([0., 5.], 1, 0., [0., 4.],  6, 5.),
       ([0., 4.], 4, 0., [1., 4.],  7, 6.),
       ([1., 4.], 1, 0., [1., 3.],  8, 5.),
       ([1., 3.], 4, 0., [2., 3.],  9, 6.),
       ([2., 3.], 4, 0., [3., 3.], 10, 5.),
       ([3., 3.], 4, 0., [4., 3.], 11, 4.),
       ([4., 3.], 1, 0., [4., 2.], 12, 3.),
       ([4., 2.], 1, 0., [5., 2.], 13, 4.),
       ([5., 2.], 4, 0., [6., 2.], 14, 3.),
       ([6., 2.], 1, 0., [6., 1.], 15, 3.),
       ([6., 1.], 1, 0., [7., 1.], 16, 4.),
       ([7., 1.], 4, 0., [8., 1.], 17, 4.),
       ([8., 1.], 0, 0., [8., 1.], 18, 4.),
       ([8., 1.], 3, 0., [7., 1.], 19, 4.),
       ([7., 1.], 4, 0., [8., 1.], 20, 4.),
       ([8., 1.], 3, 0., [7., 1.], 21, 4.),
       ([7., 1.], 1, 1., [7., 0.

In [69]:
weights[0]

[0.7132352941176471,
 0.36282606414105434,
 0.13164275282008847,
 0.034066558251510434,
 0.006287708340180658,
 0.0008277312348312113,
 7.771755426316267e-05,
 5.204532588676245e-06,
 2.48586285712858e-07,
 8.468479182763697e-09,
 3.6061492422665387e-11,
 1.9195170816455204e-14,
 7.287408118376271e-18,
 1.973272342660357e-21,
 6.678992851535058e-26,
 1.612381306110978e-30,
 2.776242228765327e-35,
 5.9752622437758e-41,
 7.796655348684653e-46,
 7.255920717711713e-51,
 8.440860713209574e-57,
 7.003477936596074e-63,
 4.1445141020010597e-69]

In [ ]:
weights_difference[1]

array([-2.01516831e+01, -5.46309224e+01, -9.69844577e+01, -1.01159396e+02,
       -2.37452773e+03, -1.53471859e+04, -6.90936339e+04, -2.13460139e+05,
       -4.39202910e+05,  5.12160962e+05,  1.77653452e+05,  4.85730579e+04,
        1.34311349e+03,  2.64266741e+01,  3.42329083e-01,  2.97261847e-02,
        1.82655288e-03,  6.07082471e-05,  1.74411777e-05,  3.45811643e-06])

In [34]:
padded_timesteps, padded_rewards, padded_actions, padded_weights = testing.padding_IS_terms(timesteps, actions, rewards, weights)

In [35]:
padded_states, padded_weights_difference = testing.padding_states_weights_difference(states, weights_difference)
padded_state_tensors, padded_weight_diff_tensors = testing.tensorize_padded_terms(padded_states, padded_weights_difference)
states_first_tensor, states_last_tensor, gamma_last_tensor, weights_last_tensor = testing.tensorize_last_and_first_terms(states_first, states_last, gamma_last, weights_last)
IS_tensor = testing.calc_IS_terms(testing.gamma, padded_timesteps, padded_rewards, padded_weights)

<ipython-input-22-80280909a6e1>:89: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  padded_weight_diff_tensors = torch.tensor(padded_weights_difference, dtype = self.dtype)


In [36]:
padded_weights_difference

[array([3.50409230e-01, 2.31183311e-01, 9.75761946e-02, 2.77788499e-02,
        5.45997711e-03, 7.50013681e-04, 7.25130217e-05, 4.95594630e-06,
        2.40117807e-07, 8.43241769e-09, 3.60422973e-11, 1.91878834e-14,
        7.28543485e-18, 1.97320555e-21, 6.67883161e-26, 1.61235354e-30,
        2.77623625e-35, 5.97518428e-41, 7.79658279e-46, 7.25591228e-51,
        8.44085371e-57, 7.00347379e-63, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
 array([-2.01516831e+01, -5.46309224e+01, -9.69844577e+01, -1.01159396e+02,
        -2.37452773e+03, -1.53471859e+04, -6.90936339e+04, -2.13460139e+05,
        -4.39202910e+05,  5.12160962e+05,  1.77653452e+05,  4.85730579e+04,
         1.34311349e+03,  2.64266741e+01,  3.42329083e-01,  2.97261847e-02,
         1.82655288e-03,  6.07082471e-05,  1.74411777e-05,  3.45811643e-06,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,

In [39]:
states_output, states_first_output, states_last_output = testing.pass_states(model, padded_state_tensors, states_first_tensor, states_last_tensor)
sums_states_weight_diff = testing.states_weight_diff_sums(states_output, padded_weight_diff_tensors)

gamma_weights_last_tensor = testing.calc_gamma_weight_last(testing.gamma, gamma_last, weights_last)

gamma_weights_states_last_sub_states_first = testing.last_first_terms_operations(gamma_weights_last_tensor, states_last_output, states_first_output)


In [48]:
states_output[0]

tensor([[ 1.0020],
        [ 0.8911],
        [ 1.3753],
        [ 1.3137],
        [ 0.4047],
        [ 0.7766],
        [ 0.2557],
        [ 0.8898],
        [ 0.6326],
        [ 0.1699],
        [ 0.5175],
        [ 0.1025],
        [ 0.7043],
        [ 0.1501],
        [ 0.3807],
        [ 1.2766],
        [ 0.2123],
        [ 0.3554],
        [ 0.8854],
        [ 1.1399],
        [-0.5327],
        [ 1.4007],
        [ 0.0669],
        [ 0.1753],
        [ 0.0639],
        [-0.0026],
        [-0.0043],
        [ 0.1673],
        [ 0.0854],
        [ 0.0112],
        [-0.0640]], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [59]:
swd = states_output*padded_weight_diff_tensors

In [67]:
swd[0]

tensor([[ 3.5109e-01],
        [ 2.0600e-01],
        [ 1.3420e-01],
        [ 3.6493e-02],
        [ 2.2097e-03],
        [ 5.8246e-04],
        [ 1.8540e-05],
        [ 4.4097e-06],
        [ 1.5191e-07],
        [ 1.4328e-09],
        [ 1.8653e-11],
        [ 1.9673e-15],
        [ 5.1309e-18],
        [ 2.9608e-22],
        [ 2.5429e-26],
        [ 2.0583e-30],
        [ 5.8936e-36],
        [ 2.1236e-41],
        [ 6.9028e-46],
        [ 8.2713e-51],
        [-4.4964e-57],
        [ 9.8098e-63],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-0.0000e+00]], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [47]:
padded_weight_diff_tensors[0]

tensor([[3.5041e-01],
        [2.3118e-01],
        [9.7576e-02],
        [2.7779e-02],
        [5.4600e-03],
        [7.5001e-04],
        [7.2513e-05],
        [4.9559e-06],
        [2.4012e-07],
        [8.4324e-09],
        [3.6042e-11],
        [1.9188e-14],
        [7.2854e-18],
        [1.9732e-21],
        [6.6788e-26],
        [1.6124e-30],
        [2.7762e-35],
        [5.9752e-41],
        [7.7966e-46],
        [7.2559e-51],
        [8.4409e-57],
        [7.0035e-63],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00]], dtype=torch.float64)

In [50]:
padded_states[0]

[[1.0, 9.0],
 [0.0, 9.0],
 [0.0, 8.0],
 [0.0, 7.0],
 [0.0, 6.0],
 [0.0, 5.0],
 [0.0, 4.0],
 [1.0, 4.0],
 [1.0, 3.0],
 [2.0, 3.0],
 [3.0, 3.0],
 [4.0, 3.0],
 [4.0, 2.0],
 [5.0, 2.0],
 [6.0, 2.0],
 [6.0, 1.0],
 [7.0, 1.0],
 [8.0, 1.0],
 [8.0, 1.0],
 [7.0, 1.0],
 [8.0, 1.0],
 [7.0, 1.0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]

In [44]:
sums_states_weight_diff

tensor([[ 7.3060e-01],
        [-5.3051e+05],
        [ 5.5980e-01],
        [ 1.3621e-01],
        [-8.3384e+04]], dtype=torch.float64, grad_fn=<SumBackward1>)

In [ ]:
test1 = SCOPE_variance(model, 0.9, 10000, pi_b, P_pi_b, P_pi_e)

In [ ]:
s,p_s, p_w, g_w_l, s_f, s_l = test1.prepare()

# IS Play

In [125]:
def calculate_importance_weights(eval_policy, behav_policy, behavior_policies):
    """
    Calculate importance weights for behavior policies.

    Parameters:
    - eval_policy: Evaluation policy
    - behav_policy: Behavior policy
    - behavior_policies: List of behavior policies

    Returns:
    - all_weights: List of importance weights
    """
    all_weights_temp = []
    for trajectory in behavior_policies:
        cum_ratio = 1
        cumul_weights = []
        for step in trajectory:
            # eval_action_probs = get_quadrant_policy(step[0], eval_policy)
            # behav_action_probs = get_quadrant_policy(step[0], behav_policy)

            P_pi_b = behav_policy[tuple(np.append(step[0].astype(int) , (step[1],)))]
            P_pi_e = eval_policy[tuple(np.append(step[0].astype(int) , (step[1],)))]

            # ratio = (0.8*eval_action_probs[step[1]] +0.2*0.25)/ (0.8*behav_action_probs[step[1]]+0.2*0.25)
            ratio = P_pi_e/P_pi_b
            cum_ratio *= ratio
            cumul_weights.append(cum_ratio)
        all_weights_temp.append(cumul_weights)

        all_weights = [list(np.cumprod(i)) for i in all_weights_temp]

    return all_weights_temp, all_weights

In [126]:
all_weights_temp, all_weights = calculate_importance_weights(P_pi_e, P_pi_b, pi_b)

In [128]:
all_weights_temp[0]

[0.7132352941176471,
 0.5087045847750865,
 0.36282606414105434,
 0.2587803545711932,
 0.18457128230445397,
 0.1316427528200885,
 0.09389225752609254,
 0.06696727191199248,
 0.0477634218784064,
 0.03406655825151045,
 0.004258319781438806,
 0.0005322899726798508,
 0.00037964799522018767,
 0.0002707783495320456,
 3.38472936915057e-05,
 2.4141084471147448e-05,
 1.721827348309781e-05,
 2.1522841853872264e-06,
 1.304822287391006e-05,
 9.306453079185852e-06,
 1.1633066348982315e-06,
 8.297113498906504e-07,
 5.917794186720081e-07]

In [90]:
P_pi_b[2,9,0]

0.08

In [99]:
states_test = pi_b[0]['state']

In [103]:
states_test.shape

(23, 2)

In [100]:
actions_test = pi_b[0]['action']

In [102]:
actions_test.shape

(23,)

In [106]:
# Extract probability for each state-action pair
num_steps, num_states, num_actions = P_pi_b.shape
num_samples = states_test.shape[0]

# Convert states_test to integers for indexing
states_int = states_test.astype(int)

# Extract probabilities using array indexing
probs_pi_b = P_pi_b[states_int[:, 0], states_int[:, 1], actions_test]
probs_pi_e = P_pi_e[states_int[:, 0], states_int[:, 1], actions_test]

In [107]:
probs_pi_b

array([0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.68, 0.08,
       0.08, 0.68, 0.68, 0.08, 0.68, 0.68, 0.08, 0.08, 0.68, 0.08, 0.68,
       0.68])

In [108]:
probs_pi_e

array([0.485, 0.485, 0.485, 0.485, 0.485, 0.485, 0.485, 0.485, 0.485,
       0.485, 0.01 , 0.01 , 0.485, 0.485, 0.01 , 0.485, 0.485, 0.01 ,
       0.485, 0.485, 0.01 , 0.485, 0.485])

In [110]:
(probs_pi_e/probs_pi_b).cumprod(axis=-1)

array([7.13235294e-01, 5.08704585e-01, 3.62826064e-01, 2.58780355e-01,
       1.84571282e-01, 1.31642753e-01, 9.38922575e-02, 6.69672719e-02,
       4.77634219e-02, 3.40665583e-02, 4.25831978e-03, 5.32289973e-04,
       3.79647995e-04, 2.70778350e-04, 3.38472937e-05, 2.41410845e-05,
       1.72182735e-05, 2.15228419e-06, 1.30482229e-05, 9.30645308e-06,
       1.16330663e-06, 8.29711350e-07, 5.91779419e-07])

In [114]:
(probs_pi_e/probs_pi_b).prod(axis=-1)

5.917794186720081e-07

In [118]:
weights[0]

[0.7132352941176471,
 0.36282606414105434,
 0.13164275282008847,
 0.034066558251510434,
 0.006287708340180658,
 0.0008277312348312113,
 7.771755426316267e-05,
 5.204532588676245e-06,
 2.48586285712858e-07,
 8.468479182763697e-09,
 3.6061492422665387e-11,
 1.9195170816455204e-14,
 7.287408118376271e-18,
 1.973272342660357e-21,
 6.678992851535058e-26,
 1.612381306110978e-30,
 2.776242228765327e-35,
 5.9752622437758e-41,
 7.796655348684653e-46,
 7.255920717711713e-51,
 8.440860713209574e-57,
 7.003477936596074e-63,
 4.1445141020010597e-69]

In [122]:
weights

[[0.7132352941176471,
  0.36282606414105434,
  0.13164275282008847,
  0.034066558251510434,
  0.006287708340180658,
  0.0008277312348312113,
  7.771755426316267e-05,
  5.204532588676245e-06,
  2.48586285712858e-07,
  8.468479182763697e-09,
  3.6061492422665387e-11,
  1.9195170816455204e-14,
  7.287408118376271e-18,
  1.973272342660357e-21,
  6.678992851535058e-26,
  1.612381306110978e-30,
  2.776242228765327e-35,
  5.9752622437758e-41,
  7.796655348684653e-46,
  7.255920717711713e-51,
  8.440860713209574e-57,
  7.003477936596074e-63,
  4.1445141020010597e-69],
 [6.0625,
  26.214183134191178,
  80.84510557563685,
  177.82956323277526,
  278.98895939887166,
  2653.5166914163347,
  18000.702591032885,
  87094.33652773884,
  300554.4756778875,
  739757.3859855295,
  227596.42361250217,
  49942.971995499654,
  1369.9141292252755,
  26.80063802941003,
  0.37396395766511065,
  0.03163487428795419,
  0.001908689588734961,
  8.213671358375234e-05,
  2.1428466437824626e-05,
  3.9872887390228824e

In [123]:
weights_first = [trajectory[0] for trajectory in weights]


In [124]:
weights_first

[0.7132352941176471, 6.0625, 0.7132352941176471, 0.125, 0.7132352941176471]

In [95]:
states[0].astype(int)[:,0]

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 7, 8, 7])